# Predicting Monetary Policy Changes using NLP

## Step 1: scraping central banks websites

Goals of this notebook:
+ Get the links of the statements in Peru
+ Download and store the PDF of interest
+ Create a master df with the name, date and text of each statement

In [1]:
# Import packages 
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from bs4 import BeautifulSoup
from time import sleep
from random import uniform
import re
import os
import pandas as pd
import wget

def wait_between(a, b):
	arb = uniform(a, b) 
	sleep(arb)

data_path = "/home/renatovassallo/Omega/BSE/Term2/Textmining/term_paper/Data"

### Web Scraping

In [67]:
profile = webdriver.FirefoxProfile()
os.makedirs(data_path+'/statements_peru/', exist_ok=True)
profile.set_preference("browser.download.dir", data_path+'/statements_peru/')
profile.set_preference("browser.download.folderList", 2)
profile.set_preference("browser.download.manager.showWhenStarting", False)
profile.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/pdf")
profile.set_preference("pdfjs.disabled", True)

option = webdriver.FirefoxOptions()
driverService = Service('browser/geckodriver')
driver = webdriver.Firefox(service=driverService, firefox_profile=profile, options=option)

# Get the href links from html using Selenium + BeauifulSoup
main = 'https://www.bcrp.gob.pe'
driver.get(main + '/politica-monetaria/notas-informativas-del-programa-monetario')
wait_between(2,4)
soup = BeautifulSoup(driver.page_source, 'html')
driver.quit()

# Keep only official statements 
text = str(soup)
soup_clean = BeautifulSoup(text.split('<h2>2002</h2>')[0], 'html')

# Get urls
urls = []
for link in soup_clean.find_all('a', href=True):
    lenght = len(re.findall('(\/docs\/Transparencia\/Notas-Informativas\/\d{4})', link['href']))
    if lenght == 1:
        if link['href'] not in urls:
             urls.append(link['href'])

print('Number of statements processed: ', len(urls))
print('----------------------------------')
urls[:20]

/tmp/ipykernel_1240/529701010.py:3: DeprecationWarning: firefox_profile has been deprecated, please use an Options object
  profile = webdriver.FirefoxProfile()
/tmp/ipykernel_1240/529701010.py:13: DeprecationWarning: firefox_profile has been deprecated, please pass in an Options object
  driver = webdriver.Firefox(service=driverService, firefox_profile=profile, options=option)


Number of statements processed:  243
----------------------------------


['/docs/Transparencia/Notas-Informativas/2023/nota-informativa-2023-01-12-1.pdf',
 '/docs/Transparencia/Notas-Informativas/2023/nota-informativa-2023-02-09-1.pdf',
 '/docs/Transparencia/Notas-Informativas/2022/nota-informativa-2022-01-06-1.pdf',
 '/docs/Transparencia/Notas-Informativas/2022/nota-informativa-2022-02-10-1.pdf',
 '/docs/Transparencia/Notas-Informativas/2022/nota-informativa-2022-03-10-1.pdf',
 '/docs/Transparencia/Notas-Informativas/2022/nota-informativa-2022-04-07-1.pdf',
 '/docs/Transparencia/Notas-Informativas/2022/nota-informativa-2022-05-12-1.pdf',
 '/docs/Transparencia/Notas-Informativas/2022/nota-informativa-2022-06-09-1.pdf',
 '/docs/Transparencia/Notas-Informativas/2022/nota-informativa-2022-07-07-1.pdf',
 '/docs/Transparencia/Notas-Informativas/2022/nota-informativa-2022-08-11-1.pdf',
 '/docs/Transparencia/Notas-Informativas/2022/nota-informativa-2022-09-08-1.pdf',
 '/docs/Transparencia/Notas-Informativas/2022/nota-informativa-2022-10-06-1.pdf',
 '/docs/Transpar

### Download PDFs

In [ ]:
# Be careful! This line takes around 40 min (human behavior)
urls.remove('/docs/Transparencia/Notas-Informativas/2020/nota-informativa-2020-03-12-1.pdf')
for url in urls:
    url = main + url
    wget.download(url, data_path+'/statements_peru/')
    wait_between(8, 12)

### You can run from here!

In [2]:
for root, dirs, files in os.walk(data_path+'/statements_peru/'):
    print('Number of statements stored: ', len(files))
    print('----------------------------------')

Number of statements stored:  242
----------------------------------


In [3]:
# Useful functions: extract text from PDF and get month number from strings
from io import StringIO
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
import datetime

def extract_text_from_pdf(pdf_path):
    resource_manager = PDFResourceManager()
    file_ptr = open(pdf_path, 'rb')
    string_io = StringIO()
    text_converter = TextConverter(resource_manager, string_io, laparams=LAParams())
    pdf_interpreter = PDFPageInterpreter(resource_manager, text_converter)

    for page in PDFPage.get_pages(file_ptr, caching=True, check_extractable=True):
        pdf_interpreter.process_page(page)

    text = string_io.getvalue()
    file_ptr.close()
    string_io.close()

    return text

def get_month(x):
    months = {'enero': 1,'febrero': 2,'marzo': 3,'abril':4,'mayo':5,'junio':6,
         'julio':7,'agosto':8,'setiembre':9,'septiembre':9,'octubre':10,'noviembre':11,'diciembre':12}
    a = x.lower()
    try:
        ez = months[a]
        return (ez)
    except:
        raise ValueError('Not a month')

In [4]:
# Creating master df_text with a loop
df_text = []
for file in files:
    new_statement = {}
    try:
        fname = data_path + '/statements_peru/' + file
        text = extract_text_from_pdf(fname)
        text = text.strip()
        text = re.sub(r'^[\n]*', '', text) 
        text = re.sub('\\n', '', text) 
        text = re.sub('\s{2}', ' ', text) 
        text = re.sub('\s{2}', ' ', text) 
        text = re.sub('\s{2}', ' ', text) 
        text = re.sub('BANCO CENTRAL DE RESERVA DEL PERÚ ', ' ', text)
        text = re.sub(r'S\/. ','',text)
        text = re.sub(r'\s\d\.','.',text)
        text = re.sub(r'\s[a-z]\.','.',text)
        text = re.sub(r'\.\.','.',text)

        # Add new row to master df_text
        new_statement['statement'] = file
        
        # Creating feature of time
        try:
            regex_date_1 = "(\d{1,2})\s..\s(enero|febrero|marzo|abril|mayo|junio|julio|agosto|septiembre|setiembre|octubre|noviembre|diciembre)\s..\s(\d{4})"
            date_1 = list(re.findall(regex_date_1,text)[-1])
            new_statement['date_1'] = datetime.date(year=int(date_1[2]), month=get_month(date_1[1]), day=1)
        except Exception as e:
                        count =+1

        try:
            regex_date_2 = '(ENERO|FEBRERO|MARZO|ABRIL|MAYO|JUNIO|JULIO|AGOSTO|SETIEMBRE|SEPTIEMBRE|OCTUBRE|NOVIEMBRE|DICIEMBRE)\s..\s(\d{4})'
            date_2 = list(re.findall(regex_date_2,text)[0])
            new_statement['date_2'] = datetime.date(year=int(date_2[1]), month=get_month(date_2[0]), day=1)
        except Exception as e:
            new_statement['date_2'] = datetime.date(year=int(date_1[2]), month=get_month(date_1[1]), day=1)

        try:
            regex_date_3 = '(ENERO|Enero|FEBRERO|Febrero|MARZO|Marzo|ABRIL|Abril|MAYO|Mayo|JUNIO|Junio|JULIO|Julio|AGOSTO|Agosto|SETIEMBRE|SEPTIEMBRE|Setiembre|OCTUBRE|Octubre|NOVIEMBRE|Noviembre|DICIEMBRE|Diciembre)\s(\d{4})'
            date_3 = list(re.findall(regex_date_3,text)[0])
            new_statement['date'] = datetime.date(year=int(date_3[1]), month=get_month(date_3[0]), day=1)
        except Exception as e:
            if len(date_2)>0:
                  new_statement['date'] = datetime.date(year=int(date_2[1]), month=get_month(date_2[0]), day=1)
            else:
                  new_statement['date'] = datetime.date(year=int(date_1[2]), month=get_month(date_1[1]), day=1)

        # Adding the text
        new_statement['text'] = text

        # Append to the master df_text
        df_text.append(new_statement)

    except Exception as e:
                count =+1

df_text = pd.DataFrame(df_text)
df_text = df_text.drop(columns=['date_1','date_2'])
df_text = df_text.sort_values(by='date', ignore_index=True)
df_text.to_csv(data_path+'/text_data_per.csv',index=False)
df_text

,statement,date,text
0,Nota-Informativa-001-2003-BCRP.pdf,2003-01-01,Nota de Prensa DIRECTORIO DEL BCR APR...
1,Nota-Informativa-002-2003-BCRP.pdf,2003-02-01,Nota de Prensa DIRECTORIO DEL BCR APR...
2,Nota-Informativa-003-2003-BCRP.pdf,2003-03-01,Nota de Prensa DIRECTORIO DEL BCR APR...
3,Nota-Informativa-004-2003-BCRP.pdf,2003-04-01,Nota de Prensa PROGRAMA MONETARIO ...
4,Nota-Informativa-005-2003-BCRP.pdf,2003-05-01,Nota de Prensa DIRECTORIO DEL BCR ...
...,...,...,...
237,nota-informativa-2022-10-06-1.pdf,2022-10-01,NOTA INFORMATIVA PROGRAMA MONETARIO DE OCTUBRE...
238,nota-informativa-2022-11-10-1.pdf,2022-11-01,NOTA INFORMATIVA PROGRAMA MONETARIO DE NOVIEMB...
239,nota-informativa-2022-12-07-1.pdf,2022-12-01,NOTA INFORMATIVA PROGRAMA MONETARIO DE DICIEMB...
240,nota-informativa-2023-01-12-1.pdf,2023-01-01,NOTA INFORMATIVA PROGRAMA MONETARIO DE ENERO 2...


In [5]:
# Let's see some statements
for i in range(239,242):
    print('File name: ', df_text.statement[i])
    print('Date: ', df_text.date[i])
    print(df_text.text[i])
    print('-----')

File name:  nota-informativa-2022-12-07-1.pdf
Date:  2022-12-01
NOTA INFORMATIVA PROGRAMA MONETARIO DE DICIEMBRE 2022 BCRP ELEVA LA TASA DE INTERÉS DE REFERENCIA A 7,50%. El Directorio del Banco Central de Reserva del Perú acordó elevar la tasa de interés de referencia en 25 pbs. a 7,50 por ciento, continuando con los ajustes de la posición de política monetaria. Para esta decisión se consideró la siguiente información:. La tasa de inflación a doce meses aumentó de 8,28 por ciento en octubre a 8,45 por ciento en noviembre, mientras que la tasa de inflación sin alimentos y energía a doce meses se redujo de 5,72 por ciento en octubre a 5,71 por ciento en noviembre. Ambos indicadores se ubicaron por encima del límite superior del rango meta de inflación. ii. El aumento significativo de los precios internacionales de energía y alimentos desde la segunda mitad del año pasado, acentuado por los conflictos internacionales, ha conllevado a un fuerte incremento de las tasas de inflación a nivel